In [1]:
dir = "E:/DataforPractice/JapMedia/"

In [2]:
import pandas as pd

dat = pd.read_excel(dir+"data/kor_data/조선일보_2022년도 데이터.xlsx")
dat.insert(0, 'id', dat.index)
dat = dat.filter(['id','제목','본문','주요 키워드'])
dat.head()

,id,제목,본문,주요 키워드
0,0,"잡초에만 농약 분사, 숲처럼 만든 전시관… 올해 CES는 온통 초록빛","소고기를 대체하는 곰팡이 단백질, 잡초만 콕 집어 제초제를 분사하는 로봇, 나무찌꺼...","CES, 친환경, 첨단 기술, 글로벌 기업들, 소비자 환경 쇼, 국내 기업, 친환경..."
1,1,"EU, 녹색 투자에 원자력 포함… 美·日, 소형원전 공동 개발",2022년 새해 벽두부터 전 세계가 원자력 발전에 주목하고 있다. 기후변화의 주범인...,"원자력, 발전, 이산화탄소, 발생, 현실 인식, 유럽연합, EU, 녹색 투자, 그린..."
2,2,"[이영완의 사이언스카페] 괴테가 사랑한 흑맥주, 릴케가 즐긴 필스너… 기후변화로 사라질까",지난해 옥스퍼드 영어사전에 ‘치맥’이란 단어가 올라갔다. 맥주 하면 치킨이라는 말이...,"맥주, 원료, 홉, 보리 재배, 직격탄, 온난화, 극심한 가뭄, 농사, 타격, 항생..."
3,3,"작년엔 전기차, 올해는 메타버스… ETF의 질주",지난해 상장지수펀드(ETF) 시장은 새롭게 태어났다는 평가를 받는다. 그 전에는 코...,"ETF 시장, 친환경, 메타버스, 해외, 탄소배출권, 투자, 유럽, 중국, 전기차 ..."
4,4,IAEA “전세계 원전 20년내 100기 더 건설”,"탄소 중립 흐름 속에 에너지 위기까지 닥치자, 재생에너지 약점을 보완할 에너지원으로...","재생에너지, 약점, 보완, 에너지원, 원전, 주목, IAEA, 추가 건설, 제안, ..."


In [ ]:
import pandas as pd
import numpy as np
import ollama
import json
import re
import gc
import traceback
import os
import openpyxl 
import glob  # 👈 (파일 목록 검색을 위해 추가)
from typing import List, Optional

# --- 1. 기본 설정 ---
dir = "E:/DataforPractice/JapMedia/"
MODEL = "llama3:8b" # "gpt-oss:20b" 
ERROR_LOG_PATH = "translation_error_log.txt"

KOR_DATA_DIR = os.path.join(dir, "data", "kor_data")

SYSTEM_PROMPT = (
    "You are an expert translator specializing in Korean-to-English news articles. "
    "You will be given a single Korean string. "
    "Your task is to translate it into professional, journalistic English.\n"
    "- Maintain a neutral and objective tone.\n"
    "- Respond ONLY with the single, translated English string. "
    "- Do not include any other text, explanations, or JSON formatting."
)

def translate_single(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "" 

    try:
        resp = ollama.chat(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": text},
            ],
            options={"temperature": 0.1, "num_ctx": 4096},
        )
        return resp['message']['content'].strip()
    
    except Exception as e:
        print(f"    - ERROR: Translation failed for text. {e}")
        with open(ERROR_LOG_PATH, "a", encoding='utf-8') as f:
            f.write(f"Translation Error: {e}\nInput Text: {text[:50]}...\n\n")
        return "ERROR: Translation failed"

if __name__ == "__main__":
    print("--- Script Start (Processing ALL .xlsx files in folder) ---")
    print(f"Config: MODEL={MODEL}")
    print(f"Target Directory: {KOR_DATA_DIR}")

    search_path = os.path.join(KOR_DATA_DIR, "*.xlsx")
    all_excel_files = [f for f in glob.glob(search_path) if not os.path.basename(f).startswith("~$")]

    if not all_excel_files:
        print(f"!!!!!!!! ERROR: No .xlsx files found in {KOR_DATA_DIR} !!!!!!!!")
        exit()

    print(f"Found {len(all_excel_files)} files to process:")
    for f in all_excel_files:
        print(f"  - {os.path.basename(f)}")

    for input_file_path in all_excel_files:
        
        file_basename = os.path.basename(input_file_path)
        file_name_without_ext = os.path.splitext(file_basename)[0]
        
        output_file_name = f"{file_name_without_ext}_translated.csv"
        output_file_path = os.path.join(KOR_DATA_DIR, output_file_name) 

        print(f"\n========================================================")
        print(f"Processing File: {file_basename}")
        print(f"Output to: {output_file_path}")
        print(f"========================================================")

        try:
            print(f"Opening Excel file (read_only): {file_basename} ...")
            workbook = openpyxl.load_workbook(input_file_path, read_only=True)
            sheet = workbook.active
            
            print("Reading header...")
            header = [cell.value for cell in sheet[1]]
            
            translated_headers = []
            cols_to_translate = ['제목', '본문', '주요 키워드'] 
            for col in header:
                translated_headers.append(col)
                if col in cols_to_translate:
                    translated_headers.append(f"{col}_en") 
            
            print("Initializing row iterator...")
            rows_iterator = sheet.iter_rows(min_row=2, values_only=True)

        except Exception as e:
            print(f"!!!!!!!! ERROR: 엑셀 파일 로딩 중 오류 발생 !!!!!!!!")
            traceback.print_exc()
            print(f"Skipping this file: {file_basename}")
            continue 

        else:
            print("✅ Excel file loaded successfully. Starting row-by-row processing loop...")
            
            try:
                pd.DataFrame(columns=translated_headers).to_csv(output_file_path, index=False, encoding="utf-8-sig", mode='w')
                print(f"Output file header written to {output_file_path}")
            except Exception as e:
                print(f"!!!!!!!! ERROR: Output file Csv write error {e} !!!!!!!!")
                print(f"Skipping this file: {file_basename}")
                continue 
                
            row_counter = 0

            while True:
                try:
                    row = next(rows_iterator)
                    row_counter += 1
                    print(f"\n--- Processing {file_basename} - Row ID: {row_counter} ---")
                    
                    row_data = dict(zip(header, row))
                    translated_row_output = {}

                    for col_name, col_value in row_data.items():
                        translated_row_output[col_name] = col_value
                        
                        if col_name in cols_to_translate:
                            print(f"  - Translating '{col_name}'...")
                            translated_text = translate_single(col_value)
                            translated_row_output[f"{col_name}_en"] = translated_text
                    
                    output_df = pd.DataFrame([translated_row_output], columns=translated_headers)
                    output_df.to_csv(output_file_path, index=False, encoding="utf-8-sig", mode='a', header=False)
                    print(f"Row {row_counter} appended to CSV.")

                except StopIteration:
                    print(f"\n--- Reached end of file: {file_basename} ---")
                    break
                
                except Exception as e:
                    print(f"!!!!!!!! ERROR processing row {row_counter} in {file_basename} !!!!!!!!")
                    traceback.print_exc()
                    with open(ERROR_LOG_PATH, "a", encoding='utf-8') as f:
                        f.write(f"File: {file_basename}\nFailed processing row {row_counter}\nError: {e}\n\n")

                finally:
                    if 'row_data' in locals(): del row_data
                    if 'translated_row_output' in locals(): del translated_row_output
                    if 'output_df' in locals(): del output_df
                    gc.collect() 
            
            print(f"\n--- Finished file: {file_basename}. Total rows: {row_counter} ---")
    
    print(f"\n========================================================")
    print(f"--- All files processed ---")
    print(f"Check {KOR_DATA_DIR} for translated CSV files.")
    print(f"Check {ERROR_LOG_PATH} for any errors.")

--- Script Start (Processing ALL .xlsx files in folder) ---
Config: MODEL=llama3:8b
Target Directory: E:/DataforPractice/JapMedia/data\kor_data
Found 6 files to process:
  - 조선일보_2022년도 데이터.xlsx
  - 조선일보_2023년도 데이터.xlsx
  - 조선일보_2024년도 데이터.xlsx
  - 한겨레_2022년도 데이터.xlsx
  - 한겨레_2023년도 데이터.xlsx
  - 한겨레_2024년도 데이터.xlsx

Processing File: 조선일보_2022년도 데이터.xlsx
Output to: E:/DataforPractice/JapMedia/data\kor_data\조선일보_2022년도 데이터_translated.csv
Opening Excel file (read_only): 조선일보_2022년도 데이터.xlsx ...
Reading header...
Initializing row iterator...
✅ Excel file loaded successfully. Starting row-by-row processing loop...
Output file header written to E:/DataforPractice/JapMedia/data\kor_data\조선일보_2022년도 데이터_translated.csv

--- Processing 조선일보_2022년도 데이터.xlsx - Row ID: 1 ---
  - Translating '제목'...
